In [ ]:
# CELLA 1: Installa tutte le librerie necessarie
print("⏳ Installazione delle librerie in corso... (potrebbe richiedere un po' di tempo)")
!pip install yt-dlp
!pip install demucs -q
!pip install pydub -q

print("✅ Installazione completata!")

In [ ]:
# CELLA 2: Importa le librerie e definisci le funzioni (Versione aggiornata)
import os
import glob
import subprocess
import ipywidgets as widgets
from pydub import AudioSegment
from IPython.display import display, Audio, clear_output
import yt_dlp

# --- Funzioni di supporto ---

def process_and_mix(yt_url, mix_type):
    """
    Funzione principale che esegue il download, la separazione e il mixaggio.
    """
    BASE_DIR = "/content/audio_mixer"
    INPUT_DIR = os.path.join(BASE_DIR, "input")
    SEPARATED_DIR = os.path.join(BASE_DIR, "separated")
    OUTPUT_DIR = os.path.join(BASE_DIR, "output")

    # Crea le directory necessarie
    for d in [INPUT_DIR, SEPARATED_DIR, OUTPUT_DIR]:
        os.makedirs(d, exist_ok=True)

    try:
        # --- 1. Download da YouTube ---
        print(f"⬇️  Scaricando l'audio da YouTube...")
        input_audio_path = os.path.join(INPUT_DIR, "input_audio.mp3")
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': input_audio_path, # CORRETTO: Usa il percorso del file, non un URL
            'noplaylist': True,
            'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3'}],
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([yt_url])
        print("✅ Audio scaricato con successo!")

        # --- 2. Separazione con Demucs ---
        print("\n🎶 Separazione delle tracce con Demucs... (questo è il passaggio più lungo)")
        command = f'python -m demucs.separate -n htdemucs --mp3 "{input_audio_path}" -o "{SEPARATED_DIR}"'
        subprocess.run(command, shell=True, check=True, capture_output=True)

        # Trova la cartella del brano creata da demucs
        htdemucs_output_dir = os.path.join(SEPARATED_DIR, "htdemucs")
        song_folder_name = os.listdir(htdemucs_output_dir)[0]
        tracks_path = os.path.join(htdemucs_output_dir, song_folder_name)

        separated_tracks = glob.glob(os.path.join(tracks_path, "*.mp3"))
        print(f"✅ Tracce separate: {[os.path.basename(t) for t in separated_tracks]}")

        # --- 3. Creazione del Mix ---
        print(f"\n🎛️  Creazione del mix '{mix_type}'...")
        tracks_to_mix = []
        if mix_type == "Karaoke":
            # Escludi la traccia 'vocals'
            tracks_to_mix = [t for t in separated_tracks if "vocals" not in os.path.basename(t)]
        elif mix_type == "Drumless":
            # Escludi la traccia 'drums'
            tracks_to_mix = [t for t in separated_tracks if "drums" not in os.path.basename(t)]
        elif mix_type == "Drums only":
            # Includi *solo* la traccia 'drums'
            tracks_to_mix = [t for t in separated_tracks if "drums" in os.path.basename(t)]

        if not tracks_to_mix:
             # Se 'Drums only' è stato selezionato ma non c'è una traccia drums, dà un errore chiaro.
             if mix_type == "Drums only":
                 raise Exception("Traccia 'drums' non trovata da Demucs.")
             else:
                 raise Exception("Nessuna traccia trovata per il mix. Strano!")

        # Se c'è una sola traccia (es. Drums only), non c'è bisogno di mixare.
        if len(tracks_to_mix) == 1:
            final_mix = AudioSegment.from_mp3(tracks_to_mix[0])
        else:
            # Carica la prima traccia come base
            final_mix = AudioSegment.from_mp3(tracks_to_mix[0])
            # Sovrapponi le altre tracce
            for track_file in tracks_to_mix[1:]:
                segment = AudioSegment.from_mp3(track_file)
                final_mix = final_mix.overlay(segment)

        # --- 4. Esporta e mostra il risultato ---
        output_filename = f"final_mix_{mix_type.lower().replace(' ', '_')}.mp3"
        final_output_path = os.path.join(OUTPUT_DIR, output_filename)
        final_mix.export(final_output_path, format="mp3")

        clear_output(wait=True)
        print(f"🎉 Mix finale completato! 🎉")
        print(f"File salvato in: {final_output_path}")
        print("\nAscolta il risultato qui sotto:")
        display(Audio(final_output_path))

    except Exception as e:
        clear_output(wait=True) # Pulisce l'output anche in caso di errore
        print(f"❌ Si è verificato un errore: {e}")
        print("Controlla che il link di YouTube sia valido e riprova.")


# --- Creazione Interfaccia ---
url_input = widgets.Text(
    value='https://www.youtube.com/watch?v=hBvJyi3fbtA', # Valore di default
    placeholder='Incolla qui il link di YouTube',
    description='Link YouTube:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)

mix_type_selector = widgets.RadioButtons(
    options=['Karaoke', 'Drumless', 'Drums only'],
    description='Tipo di Mix:',
    style={'description_width': 'initial'},
    disabled=False
)

button = widgets.Button(
    description="Crea Mix Audio",
    button_style='success',
    tooltip='Avvia il processo',
    icon='play'
)

output_area = widgets.Output()

def on_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        if not url_input.value.strip():
            print("Per favore, inserisci un link di YouTube.")
            return
        print("🚀 Avvio del processo... Preparati!")
        process_and_mix(url_input.value, mix_type_selector.value)

button.on_click(on_button_clicked)

In [ ]:
# CELLA 3: Esegui l'applicazione
# Esegui questa cella per mostrare l'interfaccia.
# Incolla il link, scegli il tipo di mix e clicca il pulsante!

print(" AI Audio Mixer ")
print("Inserisci un link di YouTube, scegli il tipo di mix e clicca il pulsante per iniziare.")
display(url_input, mix_type_selector, button, output_area)